In [6]:
import numpy as np
def get_data():
    f = open('wine.data','r',encoding='utf-8')
    X,Y = [],[]
    for line in f.readlines():
        line = line.strip('\n').strip()
        xy = line.split(',')
        y,x= xy[0],xy[1:]
        x = [float(i) for i in x]
        X.append(x)
        Y.append(float(y))
    EX = np.mean(X,axis=0).reshape(1,-1)
    sigma = np.std(X,axis=0).reshape(1,-1)
    X = (X - EX)/sigma
    return np.array(X),np.array(Y).reshape(-1,1)

X,Y  = get_data()
print(X.shape)
print(Y.shape)
data = np.hstack((X,Y))
print(data.shape)
np.random.shuffle(data)
m,n = data.shape
i = int(m*0.7)
x_train,y_train = data[:i,:-1],data[:i,-1].reshape(-1,1)
x_test,y_test = data[i:,:-1],data[i:,-1].reshape(-1,1)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(178, 13)
(178, 1)
(178, 14)
(124, 13) (124, 1)
(54, 13) (54, 1)


In [7]:
alpha = 0.00000001
gamma = 10000
beta = 100000000

In [8]:
def get_garident(X,Y,M):
    m,n = X.shape
    d = np.zeros((n,n))
    for i in range(m):
        x_i = X[i]
        if np.dot(np.dot((x_i).T,M),(x_i)) < 0:
            d -= gamma*np.dot(x_i,(x_i).T)
        for j in range(m):
            x_j = X[j]
            if Y[i] == Y[j]:
                d += np.dot((x_i-x_j),(x_i-x_j).T)
            else:
                if 1 - np.dot(np.dot((x_i-x_j).T,M),(x_i-x_j)) > 0:
                    d -= beta*np.dot((x_i-x_j),(x_i-x_j).T)
    return d

In [9]:
def get_loss(X,Y,M):
    m,n = X.shape
    l = 0
    for i in range(m):
        x_i = X[i]
        l += gamma*max(0,-1*np.dot(np.dot((x_i).T,M),(x_i)))
        for j in range(m):
            x_j = X[j]
            if Y[i] == Y[j]:
                l += np.dot(np.dot((x_i-x_j).T,M),(x_i-x_j))
            else:
                 l +=beta*max(0,1 - np.dot(np.dot((x_i-x_j).T,M),(x_i-x_j)))
    return l
    

In [10]:
def sgd(X,Y,iters):
    m,n = X.shape
    M = np.identity(X.shape[1])
    iter = 0
    mini_batch = 32
    while iter < iters:
        print('iter: ',iter,'loss = ',get_loss(X,Y,M))
        idxs = np.arange(m)
        np.random.shuffle(idxs)
        for i in range(0,m,mini_batch):
            if i + mini_batch < m:
                idxs = idxs[i:i+mini_batch]
            else:
                idxs = idxs[i:]
            x_batch,y_batch = X[idxs],Y[idxs]
            d = get_garident(x_batch,y_batch,M)
            M = M - alpha*d
        iter += 1
    print(M)
    return M

    

In [11]:
alpha = 0.00000001
gamma = 10000
beta = 1000000
M = sgd(X,Y,1000)

iter:  0 loss =  163754.85354397073
iter:  1 loss =  163743.02177805232
iter:  2 loss =  163720.99065732374
iter:  3 loss =  163696.2502927888
iter:  4 loss =  163679.09462084115
iter:  5 loss =  163659.95200815194
iter:  6 loss =  163646.21678748785
iter:  7 loss =  163633.0166294185
iter:  8 loss =  163620.0688482728
iter:  9 loss =  163606.15155377344
iter:  10 loss =  163594.17805165483
iter:  11 loss =  163578.0840835806
iter:  12 loss =  163566.4280157942
iter:  13 loss =  163548.4515639453
iter:  14 loss =  163530.15376101242
iter:  15 loss =  163518.4631181268
iter:  16 loss =  163504.66060246923
iter:  17 loss =  163491.01551377258
iter:  18 loss =  163475.04047955608
iter:  19 loss =  163463.9673662703
iter:  20 loss =  163451.01632004176
iter:  21 loss =  163435.9849992267
iter:  22 loss =  163419.30255544677
iter:  23 loss =  163401.89198241138
iter:  24 loss =  163388.31935502565
iter:  25 loss =  163368.640781683
iter:  26 loss =  163355.29049736809
iter:  27 loss =  1633

In [12]:
m,_ = M.shape
for i in range(m):
    if np.linalg.det(M[:i,:i]) < 0:
        print('not postive')